<a href="https://colab.research.google.com/github/syedareehaquasar/SUE---Safe-User-Experience/blob/master/PhishingWebsiteDetectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Phishing Website Detection ML model training**

Designing model for creating phishing detection browser extension

**Loading Data**